In [1]:
# https://docs.google.com/document/d/1e9oakyEftdHp4zGz2F0WP-_X7GEzEKYGaPvu1SguuCE/edit

import pandas as pd
import numpy as np
import matplotlib as plt
from datetime import datetime
from datetime import date as dt
from datetime import timedelta as td



# https://pypi.org/project/yfinance/#description
import yfinance as yf

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

# Transformations:

### Transform 1: Group by industry

Calculate:
* Market value per industry (dollar amount) for the day
* % change of market value from previous day for each industry

(market value = adjusted close * volume)

In [2]:
portfolio2 = ['MSFT', 'AAPL', 'IBM', 'WMT', 'SHOP', 'LWLG', 'ALB', 'LYV', 'GOOGL', 'TTGT', 'TSLA', 'GME', 'AMZN', 'TGT', 'COST', 'COKE','TPL', 'BX', 'MORN', 'CBRE', 
            'NVDA', 'AMD', 'NEE']
portfolio = ['MSFT', 'WMT', 'IBM', 'WMT', 'SHOP'] # reduced portfolio to make script quicker (pre prod, less api calls)

### Pull industry for each ticker (for eventual aggregation) as well as current market cap (updates daily)

In [3]:
%%time

# today's date
today = dt.today().strftime("%d/%m/%Y")

# hash the entirety of the api call data for each stock
company_info = {i: yf.Ticker(i).info for i in portfolio}

# pull the respective data we'd like from the hash (instead of calling the api twice for each stock)
industry_cap_dict = {i: [today, company_info[i]['sector'], company_info[i]['marketCap']] for i in portfolio}
    

industry_cap_dict

CPU times: user 1.88 s, sys: 85.1 ms, total: 1.97 s
Wall time: 21.9 s


{'MSFT': ['01/03/2022', 'Technology', 2212687511552],
 'WMT': ['01/03/2022', 'Consumer Defensive', 376432754688],
 'IBM': ['01/03/2022', 'Technology', 108634701824],
 'SHOP': ['01/03/2022', 'Technology', 86003597312]}

In [4]:
# transform dictionary into dataframe
industry_cap_df = pd.DataFrame.from_dict(industry_cap_dict,orient='index')

# add the stock tickers, currently in index, to a column named 'company'
industry_cap_df.insert(1, 'company', industry_cap_df.index)

# move the 'date' column to the index
industry_cap_df.index = industry_cap_df.iloc[:,0]

# drop the 'date' column
industry_cap_df.drop(industry_cap_df.columns[0], axis=1, inplace=True)

# remove the title for the index column
industry_cap_df.index.name = None

# rename the remaining columns
pd.DataFrame(industry_cap_df).rename(columns={1: 'sector', 2: 'market_cap'}, inplace=True)

# Data to be used for metrics (market cap by industry)

In [5]:
industry_agg = industry_cap_df.groupby(['sector']).sum()
industry_agg

,market_cap
sector,
Consumer Defensive,376432754688
Technology,2407325810688


# Data to be used for notification purposes

In [6]:
# # set date to yesterday
# yesterday = datetime.now() - td(1)
# yesterday = datetime.strftime(yesterday, '%Y-%m-%d')

# financial_info_df = yf.download(portfolio, yesterday, group_by='Ticker')
# financial_info_df = financial_info_df.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1) # Trenton McKinney, https://stackoverflow.com/questions/63107594/how-to-deal-with-multi-level-column-names-downloaded-with-yfinance/63107801#63107801
# financial_info_df = financial_info_df[['Ticker','Adj Close']]
# financial_info_df

In [7]:
# set date to yesterday
yesterday = datetime.now() - td(4)
yesterday = datetime.strftime(yesterday, '%Y-%m-%d')

financial_info_df = yf.download(portfolio, yesterday, group_by='Ticker')
financial_info_df = financial_info_df.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1) # Trenton McKinney, https://stackoverflow.com/questions/63107594/how-to-deal-with-multi-level-column-names-downloaded-with-yfinance/63107801#63107801
financial_info_df = financial_info_df[['Ticker','Adj Close']]

# last two business days for each stock
x = len(portfolio) * 2
financial_info_df = financial_info_df[-x:]

financial_info_df

[*********************100%***********************]  4 of 4 completed


,Ticker,Adj Close
Date,,
2022-02-25,SHOP,676.950012
2022-02-25,WMT,136.380005
2022-02-28,IBM,122.510002
2022-02-28,MSFT,298.790009
2022-02-28,SHOP,694.260010
2022-02-28,WMT,135.160004
2022-03-01,IBM,121.080002
2022-03-01,MSFT,295.109985
2022-03-01,SHOP,687.270020


In [8]:
%%time
#create unique list of names
UniqueNames = portfolio

#create a data frame dictionary to store your data frames
DataFrameDict = {elem : pd.DataFrame for elem in UniqueNames}

for key in DataFrameDict.keys():
    DataFrameDict[key] = financial_info_df[:][financial_info_df.Ticker == key]

CPU times: user 2.06 ms, sys: 169 µs, total: 2.23 ms
Wall time: 2.23 ms


In [9]:
for stock in portfolio:
    DataFrameDict[stock]['percent_change'] = DataFrameDict[stock]['Adj Close'].pct_change()
    
    # set notification for threshold
    # if greater than 1%
    if abs(DataFrameDict[stock]['percent_change'][-1]) > 0.01:
        print(today, stock, DataFrameDict[stock]['percent_change'][-1])

01/03/2022 MSFT -0.012316419853798721
01/03/2022 IBM -0.011672518816754462
01/03/2022 SHOP -0.010068259925751333


In [10]:
DataFrameDict

{'MSFT':            Ticker   Adj Close  percent_change
 Date                                         
 2022-02-28   MSFT  298.790009             NaN
 2022-03-01   MSFT  295.109985       -0.012316,
 'WMT':            Ticker   Adj Close  percent_change
 Date                                         
 2022-02-25    WMT  136.380005             NaN
 2022-02-28    WMT  135.160004       -0.008946
 2022-03-01    WMT  135.706207        0.004041,
 'IBM':            Ticker   Adj Close  percent_change
 Date                                         
 2022-02-28    IBM  122.510002             NaN
 2022-03-01    IBM  121.080002       -0.011673,
 'SHOP':            Ticker   Adj Close  percent_change
 Date                                         
 2022-02-25   SHOP  676.950012             NaN
 2022-02-28   SHOP  694.260010        0.025571
 2022-03-01   SHOP  687.270020       -0.010068}